In [5]:
import os

In [6]:
%pwd


'd:\\Text-Summarizer\\research'

In [7]:
os.chdir('../')
%pwd

'd:\\Text-Summarizer'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class dataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from textSummarizer.constants import configFilePath,paramsFilePath
from textSummarizer.utils.common import read_yaml,createDir

In [10]:
class configurationManager:
    def __init__(
        self,
        config_FilePath = configFilePath,
        params_FilePath = paramsFilePath):
        
        self.config = read_yaml(config_FilePath)
        self.params = read_yaml(params_FilePath)
        
        createDir([self.config.artifacts_root])
    
    def getDataIngestionConfig(self) -> dataIngestionConfig:
        config = self.config.data_ingestion
        createDir([config.root_dir])
        
        data_ingestion_config = dataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import getFileSize

In [12]:
class dataIngestion:
    def __init__(self,config:dataIngestionConfig):
        self.config = config
    def downloadFiles(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size : {getFileSize(Path(self.config.local_data_file))}")
    
    def extractZip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = configurationManager()
    data_ingestion_config = config.getDataIngestionConfig()
    data_ingestion = dataIngestion(config=data_ingestion_config)
    data_ingestion.downloadFiles()
    data_ingestion.extractZip()
except Exception as e:
    raise e

[2024-08-16 22:31:45,453: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-16 22:31:45,462: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-16 22:31:45,462: INFO: common: Created director at: artifacts]
[2024-08-16 22:31:45,467: INFO: common: Created director at: artifacts/data_ingestion]
[2024-08-16 22:32:48,196: INFO: 4049229976: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F72C:B37EE:3B50:4A08:66BF85FD
Accept-Ranges: bytes
Date: Fri, 16 Aug 2024 17:01:50 GMT
Via: 1.1 varnish
X-Served-By: cache-del21750-DEL
X-Cache: MISS
X-C